In [ ]:
import os
import sys
import session_info
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import anndata as ad
import mudata as mu
import hdf5plugin

import pertpy
sccoda = pertpy.tl.Sccoda()

# Add repo path to sys path (allows to access scripts and metadata from repo)
repo_path = '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis'
sys.path.insert(1, repo_path) 
sys.path.insert(2, '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts')

# Autoreload custom scripts
%load_ext autoreload
%autoreload 2

# Define paths
plots_path = f'{repo_path}/plots/'
data_path = f'{repo_path}/data/'
model_path = os.path.join(repo_path, 'models')
general_data_path = '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/data'

print('Dir for plots: {}'.format(plots_path))
print('Dir for data: {}'.format(data_path))

# Formatting
from matplotlib import font_manager
font_manager.fontManager.addfont("/nfs/team205/ny1/ThymusSpatialAtlas/software/Arial.ttf")
plt.style.use('/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts/plotting/thyAgeing.mplstyle')

# Import custom scripts
from utils import get_latest_version,update_obs,freq_by_donor
from anno_levels import get_ct_levels, get_ct_palette, age_group_levels, age_group_palette
from plotting.utils import plot_grouped_boxplot, calc_figsize

In [ ]:
# Load adata
object_version = 'v5_2025-04-03'
adata = ad.read_h5ad(f'{general_data_path}/objects/rna/thyAgeing_all_scvi_{object_version}.zarr')

# Add new annotations to adata
ct_anno = pd.read_csv(f'{general_data_path}/objects/rna/thyAgeing_all_scvi_v4_2025-02-04_curatedAnno_v10.csv', index_col = 0)
adata.obs = adata.obs.join(ct_anno, how = 'left')
adata = adata[(adata.obs['anno_status'] == 'include')]

# Update metadata
latest_meta_path = get_latest_version(dir = f'{general_data_path}/metadata', file_prefix='Thymus_ageing_metadata')
latest_meta = pd.read_excel(latest_meta_path)
update_obs(adata, latest_meta, on = 'index', ignore_warning = True)

latest_ct_levels_path = get_latest_version(dir = f'{general_data_path}/curated', file_prefix='thyAgeing_full_curatedAnno')
latest_ct_levels = pd.read_excel(latest_ct_levels_path)
update_obs(adata, latest_ct_levels[['taa_l5', 'taa_l2_v2']], on = 'taa_l5', add_cols = ['taa_l2_v2'])

adata

In [ ]:
# Define columns
col_cell_type_broad = 'taa_l2_v2'
col_cell_type_fine = 'taa_l4'
col_cell_type_broad_levels = get_ct_levels(col_cell_type_broad)
col_cell_type_fine_levels = get_ct_levels(col_cell_type_fine)
col_age_group = 'age_group'
col_age_group_levels = eval(f'{col_age_group}_levels')

col_cell_type_broad_levels.remove('DC')
col_cell_type_broad_levels.remove('TEC-EMT')

In [ ]:
col_cell_type_broad_levels

In [ ]:
covs = ["sex", "age_group", "chemistry_simple"]
adata.obs['batch'] = adata.obs[['donor'] + covs].astype(str).agg('_'.join, axis=1)
adata.obs['age_group'] = pd.Categorical(adata.obs['age_group'], categories=col_age_group_levels, ordered=True)

sccoda_data = sccoda.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier=col_cell_type_broad,
    sample_identifier="batch",
    covariate_obs=covs,
)
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
sccoda_data = sccoda.prepare(
    sccoda_data,
    #modality_key="coda_salm",
    formula="sex + age_group + chemistry_simple",
    reference_cell_type="automatic",
)
sccoda_data["coda"]

In [ ]:
# Run MCMC
sccoda.run_nuts(sccoda_data)
sccoda_data["coda"]

In [ ]:
sccoda.summary(sccoda_data, modality_key="coda")

In [ ]:
effect_df = sccoda.get_effect_df(sccoda_data, modality_key="coda")
effect_df.to_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_scvi_{object_version}_sccoda_effects.csv')
effect_df

In [ ]:
np.setdiff1d(col_cell_type_broad_levels, effect_df['Cell Type'].unique())

## Plotting

In [ ]:
effect_df = pd.read_csv(f'{data_path}/analyses/freqAnalysis/thyAgeing_all_scvi_{object_version}_sccoda_effects.csv', index_col=0).reset_index()
effect_df['Covariate'].unique()

In [ ]:
np.setdiff1d(col_cell_type_broad_levels, effect_df['Cell Type'].unique())

In [ ]:
# Prepare data for plotting
effect_df = effect_df.loc[effect_df['Covariate'].isin(['age_groupT.paed', 'age_groupT.adult'])]
effect_df['Covariate'] = effect_df['Covariate'].map({'age_groupT.paed': 'Late paed', 'age_groupT.adult': 'Adult'})
effect_df['Covariate'] = pd.Categorical(effect_df['Covariate'], categories=['Late paed', 'Adult'], ordered=True)
effect_df['Cell Type'] = pd.Categorical(effect_df['Cell Type'], categories=col_cell_type_broad_levels, ordered=True)
effect_df['abs_logFC'] = effect_df['log2-fold change'].abs()

In [ ]:
from plotting.utils import thyAgeing_colors
import matplotlib.colors as mcolors

plt.figure(figsize=calc_figsize(height=30, width=160))
max_abs = np.abs(effect_df['log2-fold change']).max()
vmin, vmax, vcenter = -max_abs, max_abs, 0
normalize = mcolors.TwoSlopeNorm(vcenter=vcenter, vmin=vmin, vmax=vmax)
cmap = sns.blend_palette([thyAgeing_colors['teal'], 'white', thyAgeing_colors['orange']], as_cmap=True)

sizenorm = plt.Normalize(vmin=0, vmax=effect_df['Inclusion probability'].max())

p = sns.scatterplot(
    data=effect_df,
    y='Covariate', x='Cell Type', hue='log2-fold change',
    palette=cmap, hue_norm=normalize, size='Inclusion probability', 
    sizes=(0, 70), size_norm=sizenorm,
    legend=False
)
p.set_ylabel('Comparison')
p.set_xlabel('Cell population')
p.set_xlim(-0.5, len(effect_df['Cell Type'].cat.categories)-0.5)
p.set_ylim(1.5, -0.5)  # Reverse y-axis
p.set_xticklabels(p.get_xticklabels(), rotation=90, va='top', ha='center')
#p.set_yticklabels(group_labels[::-1], rotation=0, va='center', ha='right')

sm = plt.cm.ScalarMappable(cmap=cmap, norm=normalize)
sm.set_array([])
cbar = p.figure.colorbar(sm, ax=p.axes, orientation='vertical', label='logFC', pad=0.01, aspect=5)
plt.tight_layout()

# Add legend for dot size
for logfc in [0.2, 0.5, 0.8]:
    size = sizenorm(logfc) * (70 - 1) + 1
    plt.scatter([], [], s=size, c='gray', alpha=0.5, label=f'{logfc}')
p.legend(title='Inclusion prob', loc='upper left', bbox_to_anchor=(1.08, 1.1), frameon=False)

plt.savefig(f'{plots_path}/freqAnalysis/thyAgeing_all_taa_l2_v2_sccoda_logFcCellTypes_adult_infant_heatmap_pub.pdf')
plt.show()


In [ ]:
effect_df['Cell Type'].unique()

In [ ]:
effect_df